## Prepare Dataset

In [3]:
! pip install -q accelerate peft lightning nltk

In [4]:
! pip install -U bitsandbytes

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# download dataset

import transformers

from datasets import load_dataset

DATA_REPO_ID = "tanganke/stanford_cars"



train_dataset = load_dataset(DATA_REPO_ID, split="train")

test_dataset  = load_dataset(DATA_REPO_ID, split="test")
# valid_dataset = load_dataset(DATA_REPO_ID, split="validation")


README.md:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/504M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/485M [00:00<?, ?B/s]

test-00000-of-00002.parquet:   0%|          | 0.00/513M [00:00<?, ?B/s]

test-00001-of-00002.parquet:   0%|          | 0.00/474M [00:00<?, ?B/s]

contrast-00000-of-00001.parquet:   0%|          | 0.00/347M [00:00<?, ?B/s]

In [4]:
# prepare collate function

from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("google/paligemma-3b-pt-224")

def train_collate_fn(samples):

    images = [sample["image"] for sample in samples]

    texts = ["What is the model of a car in the image?" for sample in samples]

    labels = [sample["label"] for sample in samples]



    # define a processor that handle max_length 128(not including the number of image tokens)

    inputs = processor(text=texts, images=images, suffix=labels, return_tensors="pt", 

                      padding=True, truncation=True, max_length=128)



    input_ids = inputs["input_ids"]

    token_type_ids = inputs["token_type_ids"]

    attention_mask = inputs["attention_mask"]

    pixel_values = inputs["pixel_values"]

    labels = inputs["labels"]



    return input_ids, token_type_ids, attention_mask, pixel_values, labels



def test_collate_fn(samples):

    images = [sample["image"] for sample in samples]

    texts = ["What is the model of a car in the image?" for sample in samples]

    labels = [sample["label"] for sample in samples]



    # define a processor that handle max_length 128(not including the number of image tokens)

    inputs = processor(text=texts, images=images, return_tensors="pt", 

                      padding=True, truncation=True, max_length=128)



    input_ids = inputs["input_ids"]

    attention_mask = inputs["attention_mask"]

    pixel_values = inputs["pixel_values"]



    return input_ids, attention_mask, pixel_values, labels

    

preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

## Prepare Quantization Settings

In [17]:
# Using bitsandbytes library, setting the quantization

from transformers import BitsAndBytesConfig

import torch



bnb_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_compute_dtype=torch.bfloat16, # convert float32 to bf16 to speed up computation

    bnb_4bit_quant_type="nf4", # NF4 is 4-bit data type from QLoRA paper

)

## Prepare LoRA Settings

In [8]:
from peft import LoraConfig



lora_config = LoraConfig(

    r=4, # set the low-rank as 4 

    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], #modules that we are going to apply LoRA adapter

    task_type="CAUSAL_LM", # type of model. PaliGemma is causal language model

)

## Prepare pytorch-lightning Trainer

In [15]:
import lightning as L

from transformers import AutoProcessor

import torch 

from nltk import edit_distance

from torch.utils.data import DataLoader





class PaliGemma_Finetuned_Model(L.LightningModule):

    def __init__(self, config, model, processor):

        self.model = model 

        self.processor = processor

        self.config = config



        self.batch_size = config.get("batch_size")



        self.train_losses = []

        self.val_losses = []

        self.val_scores = []



    def training_step(self, batch, batch_idx):

        input_ids, token_type_ids, attention_mask, pixel_values, labels = batch



        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,

                            pixel_values=pixel_values, labels=labels)



        loss = outputs.loss



        self.train_losses.append(loss.item())



        self.log("train_loss", loss)

        return loss



    def validation_step(self, batch, batch_idx):

        input_ids, attention_mask, pixel_values, labels = batch



        generated_ids = self.model.generate(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values, max_new_tokens=80)

        scores = []

        

        for pred, label in zip(predictions, labels):

            score = edit_distance(pred, label) / max(len(pred), len(label))

            self.val_scores.append(score)

            scores.append(score)

        self.log("val_edit_distance", np.mean(scores), on_epoch=True)



        return scores



    def configure_optimizers(self):

        optimizer = torch.optim.AdamW(self.parameters(), lr=config.get("lr", 3e-4))

        return optimizer



    def train_dataloader(self):

        return DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, collate_fn=train_collate_fn, num_workers=3)

    def val_dataloader(self):

        return DataLoader(test_dataset, batch_size=self.batch_size, shuffle=True, collate_fn=test_collate_fn, num_workers=3)

        

## Define the model 

In [18]:
from transformers import PaliGemmaForConditionalGeneration

from peft import get_peft_model



model = PaliGemmaForConditionalGeneration.from_pretrained("google/paligemma-3b-pt-224", quantization_config=bnb_config)

model = get_peft_model(model, lora_config)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
model.print_trainable_parameters()

## Setup Training Configurations

In [ ]:
config = config = {"max_epochs": EPOCHS,

          "check_val_every_n_epoch": 1,

          "gradient_clip_val": 1.0,

          "accumulate_grad_batches": 8,

          "lr": 3e-4,

          "batch_size": 2,

          "seed":1234,

          "num_nodes": 1,

          "warmup_steps": 50,

          "result_path": "./result",

          "verbose": True,

}



model_module = PaliGemma_Finetuned_Model(config, processor, model)

## Define Callbacks

In [ ]:
from lightning.pytorch.callbacks import Callback

from lightning.pytorch.callbacks.early_stopping import EarlyStopping

from huggingface_hub import HfApi



api = HfApi()



FINETUNED_MODEL_ID="ball1433/PaliGemma-StanfordCars-finetuned"



class Print_TrainValidation_ResultCallback(Callback):

    def on_train_epoch_end(self, trainer, pl_module):

        # print the average of training loss 

        print(f'Average Training Loss: {np.mean(pl_module.train_losses)}')



        # print the average of edit distance score

        print(f'Average Validation Score: {np.mean(pl_module.val_scores)}')



        # reset the list

        pl_module.train_losses = []

        pl_module.val_scores = []





class PushToHubCallback(Callback):

    def on_train_epoch_end(self, trainer, pl_module):

        print(f"Pushing model to the hub, epoch {trainer.current_epoch}")

        pl_module.model.push_to_hub(FINETUNED_MODEL_ID,

                                    commit_message=f"Training in progress, epoch {trainer.current_epoch}")



    def on_train_end(self, trainer, pl_module):

        print(f"Pushing model to the hub after training")

        pl_module.processor.push_to_hub(FINETUNED_MODEL_ID,

                                    commit_message=f"Training done")

        pl_module.model.push_to_hub(FINETUNED_MODEL_ID,

                                    commit_message=f"Training done")



early_stop_callback = EarlyStopping(monitor="val_edit_distance", patience=20, verbose=False, mode="min")

## Training

In [ ]:
# define trainer 

trainer = L.Trainer(

        devices=-1, 

        accelerator="auto",

        max_epochs=config.get("max_epochs"),

        accumulate_grad_batches=config.get("accumulate_grad_batches"),

        check_val_every_n_epoch=config.get("check_val_every_n_epoch"),

        gradient_clip_val=config.get("gradient_clip_val"),

        precision="16-mixed",

        limit_val_batches=1.0,

        num_sanity_val_steps=2,

        callbacks=[PushToHubCallback(), Print_TrainValidation_ResultCallback(), early_stop_callback],

)



trainer.fit(model_module)

## Inference